# RAG 实现

## 加载文档

### 从文件读取，并实现拆分

In [1]:
import os
os.getcwd()

'/Users/xuehongwei/github/illufly/notes/core'

In [1]:
from illufly.importer import MarkdownFileImporter

t = MarkdownFileImporter(dir="./", chunk_size=550)
t(verbose=True)
t.documents

files ['./docs/设计理念.md', './docs/推理模式.md']
txt # illufly 设计理念介绍...
start split ...
chunks start >>
chunks end >>
txt # 常见的智能体推理模式和 illufly 的实现...
start split ...
chunks start >>
chunks end >>


[Document(text='# illufly 设计理念介绍...', metadata='{"source": "./docs/设计理念.md"}'),
 Document(text='一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：...', metadata='{"source": "./docs/设计理念.md"}'),
 Document(text='现在，要求你准确记住这些类的名字以及该从哪里引用，就开始慢慢形成挑战了。...', metadata='{"source": "./docs/设计理念.md"}'),
 Document(text=''role': 'user',...', metadata='{"source": "./docs/设计理念.md"}'),
 Document(text='# 常见的智能体推理模式和 illufly 的实现...', metadata='{"source": "./docs/推理模式.md"}'),
 Document(text='REWOO的全称是Reason without Observation，旨在通过以下方式改进ReACT风格的Agent架构：...', metadata='{"source": "./docs/推理模式.md"}'),
 Document(text='- 重规划器Replanner：根据执行情况和反馈调整计划。...', metadata='{"source": "./docs/推理模式.md"}'),
 Document(text='Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...', metadata='{"source": "./docs/推理模式.md"}'),
 Document(text='- **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...', metadata='{"source": "./docs/推理模式.md"}')]

In [12]:
print(t.documents[1].text)

一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：

```python
[
    SystemMessage('你是一个AI助手'),
    UserMessage('你好'),
    AIMessage('有什么可以帮你')
]
```

然后要求开发者尽量使用已经创建的 XXMessage 类来封装所有关于消息格式的功能。
到目前为止，这看起来很不错，也完全符合一般的设计原则。

但问题会逐渐显现。

首先是类定义的体系，仅仅上面几个类是不够的。例如：

- 你一定需要基类，比如：`BaseMessage`
- 也许你需要区分出工具消息，比如： `ToolMessage`？但这样也许不太够，因为要区分大模型返回的和工具执行的，也许是这样：`ToolCallMessage` 和 `ToolRespMessage`
- 如果你要区分携带部分信息的消息，可能还要增加 `UserMessageChunk`、`AIMessageChunk`，以及`ToolCallMessageChunk` 和 `ToolRespMessageChunk`
- 多模态的能力中图片的消息该如何定义？是否要增加 `ImageMessage`、`AudioMessage`、`VideoMessage`的定义？以及对应的 `ImageMessageChunk`、`AudioMessageChunk`、`VideoMessageChunk`？

有了这些类，你要先记住才能开始做其他的，例如：
```python
from xxxxxx.xxxxxx.messages import UserMessage, AIMessage, ImageMessage, VideoMessage ...
```

现在，要求你准确记住这些类的名字以及该从哪里引用，就开始慢慢形成挑战了。

...

类似的事情会在很多地方发生。

你的初衷是，通过这些新类的定义来简化工作，但是现在不得不记住很多新东西。

### illufly 中的解决方案

```python
[
    {
        'role': 'system',
        'content': '你是一个AI助手'
    },
    {


## 向量数据库

### 预先从 Embeddings 构建文档

In [1]:
from illufly.embeddings import DashScopeEmbeddings
from illufly.importer import MarkdownFileImporter
from illufly.vectordb import FaissDB

# 导入文档
local = MarkdownFileImporter(dir="./", chunk_size=550)
local(verbose=True)

# 文档嵌入
emb = DashScopeEmbeddings()
emb(local.documents, verbose=True)

# 文档入库
faiss = FaissDB(emb, train=False)

# 查询
faiss("智能体有哪些类型？", verbose=True)

  0s [INFO] [0.706] ./docs/推理模式.md: - **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...
  0s [INFO] [0.867] ./docs/推理模式.md: # 常见的智能体推理模式和 illufly 的实现...
  0s [INFO] [0.934] ./docs/设计理念.md: 'role': 'user',...
  0s [INFO] [0.951] ./docs/推理模式.md: Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...
  0s [INFO] [0.987] ./docs/设计理念.md: # illufly 设计理念介绍...


[(0.70617646,
  Document(text='- **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.06955220550298691, -0.040011003613471985, -0.03216838...')),
 (0.86662793,
  Document(text='# 常见的智能体推理模式和 illufly 的实现...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.03823075070977211, 0.025731494650244713, 0.0117180533...')),
 (0.9336294,
  Document(text=''role': 'user',...', metadata='{"source": "./docs/设计理念.md", "embeddings": [-0.061874520033597946, 0.01978103630244732, -0.015418602...')),
 (0.9512944,
  Document(text='Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.04360850527882576, -0.003100050613284111, -0.01595692...')),
 (0.98729813,
  Document(text='# illufly 设计理念介绍...', metadata='{"source": "./docs/设计理念.md", "embeddings": [-0.09724362939596176, 0.012529165484011173, -0.007420137...'))]

### Importer + Embeddings + VectorDB 

In [2]:
from illufly.embeddings import DashScopeEmbeddings
from illufly.vectordb import FaissDB

# 文档嵌入
db = FaissDB(DashScopeEmbeddings())
db.add(["普鸿是一家做数字消防业务的公司"])
db.add(["幻蝶是一家AI技术公司"])

# 查询
db("普鸿是啥？", top_k=5, verbose=True)

  0s [INFO] [0.487] unknown: 普鸿是一家做数字消防业务的公司
  0s [INFO] [1.348] unknown: 幻蝶是一家AI技术公司


[(0.48679453,
  Document(text='普鸿是一家做数字消防业务的公司', metadata='{"source": "unknown", "embeddings": [-0.09413302689790726, 0.01354235503822565, -0.03795371949672699...')),
 (1.3482788,
  Document(text='幻蝶是一家AI技术公司', metadata='{"source": "unknown", "embeddings": [-0.08771800994873047, 0.011481866240501404, 0.00165668292902410...'))]

In [3]:
from illufly.embeddings import DashScopeEmbeddings
from illufly.vectordb import FaissDB

# 文档嵌入
db = FaissDB(DashScopeEmbeddings())
db.add_files("./", chunk_size=550)
# 查询
db("智能体是啥？", top_k=5, verbose=True)

  0s [INFO] [0.680] ./docs/推理模式.md: - **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...
  0s [INFO] [0.953] ./docs/推理模式.md: # 常见的智能体推理模式和 illufly 的实现...
  0s [INFO] [0.960] ./docs/设计理念.md: 'role': 'user',...
  0s [INFO] [1.031] ./docs/推理模式.md: Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...
  0s [INFO] [1.038] ./docs/设计理念.md: # illufly 设计理念介绍...


[(0.67994213,
  Document(text='- **自我迭代的改进**：自动收集智能体的反思能力和规划能力的评测依据和微调依据，确保智能体可以不断优化和提升。...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.06955220550298691, -0.040011003613471985, -0.03216838...')),
 (0.95295113,
  Document(text='# 常见的智能体推理模式和 illufly 的实现...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.03823075070977211, 0.025731494650244713, 0.0117180533...')),
 (0.9598516,
  Document(text=''role': 'user',...', metadata='{"source": "./docs/设计理念.md", "embeddings": [-0.061874520033597946, 0.01978103630244732, -0.015418602...')),
 (1.0309153,
  Document(text='Self-Discover由Google研究人员提出，允许大型语言模型在没有明确标签情况下，自主选择并组合原子推理模块，生成推理结构。包含两个阶段：...', metadata='{"source": "./docs/推理模式.md", "embeddings": [-0.04360850527882576, -0.003100050613284111, -0.01595692...')),
 (1.0380639,
  Document(text='# illufly 设计理念介绍...', metadata='{"source": "./docs/设计理念.md", "embeddings": [-0.09724362939596176, 0.012529165484011173, -0.007420137...'))]

## 结合 ChatAgent 构建 RAG

In [1]:
# 创建一个工具
from illufly.types import ToolAgent, Template, TextBlock
from illufly.embeddings import DashScopeEmbeddings
from illufly.vectordb import FaissDB
from illufly.chat import ChatQwen

# 文档嵌入
faiss_db = FaissDB(DashScopeEmbeddings(), top_k=1)
faiss_db.add_files("./", chunk_size=500)

## 声明大模型
qwen = ChatQwen(
    knowledge=["illufly实现了多智能体结构", faiss_db],
    memory=[("system", "你是一个作家")]
)

qwen("ReAct是什么意思？", verbose=True)
qwen.memory

  0s [INFO] [0.646] ./docs/推理模式.md: # 常见的智能体推理模式和 illufly 的实现...
ReAct是“Reason and Act”的缩写，是一种结合了推理（Reasoning）与行动（Acting）的语言模型设计框架。这一概念源自于人工智能领域的研究，特别是在自然语言处理和机器学习中，用于提升AI系统的决策能力和执行效率。具体来说，ReAct框架通过模拟人类解决问题的方式，利用一个循环流程来完成任务，这个流程主要包括三个步骤：

1. **Thought（思考）**: 在这个阶段，模型基于当前的任务目标和已有的信息进行推理，形成一个或多个关于下一步应该采取什么行动的“思想”。这些思想体现了模型对当前情境的理解和对未来行动的规划。

2. **Action（行动）**: 根据上一阶段的思考结果，模型执行一个具体的行动。这可能涉及向环境提问、执行某个命令、或者操作外部系统等，目的是获取新的信息或直接推动任务进展。

3. **Observation（观察）**: 执行行动后，模型会接收并处理来自环境或系统的反馈，这成为新的观察信息。这些观察结果被整合进模型的知识库中，成为下一轮思考的基础。

通过这样的Thought-Action-Observation循环，ReAct框架使AI代理能够逐步逼近任务目标，同时保持决策过程的透明性和可解释性。这种方法不仅提高了模型处理复杂任务的能力，也便于开发者和用户理解和调试模型的行为。然而，它也面临着如输出不稳定、成本高昂及响应时间较长等挑战，这些问题在实际应用中需要通过优化算法或系统设计来缓解。



[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': 'ReAct是什么意思？'},
 {'role': 'assistant',
  'content': 'ReAct是“Reason and Act”的缩写，是一种结合了推理（Reasoning）与行动（Acting）的语言模型设计框架。这一概念源自于人工智能领域的研究，特别是在自然语言处理和机器学习中，用于提升AI系统的决策能力和执行效率。具体来说，ReAct框架通过模拟人类解决问题的方式，利用一个循环流程来完成任务，这个流程主要包括三个步骤：\n\n1. **Thought（思考）**: 在这个阶段，模型基于当前的任务目标和已有的信息进行推理，形成一个或多个关于下一步应该采取什么行动的“思想”。这些思想体现了模型对当前情境的理解和对未来行动的规划。\n\n2. **Action（行动）**: 根据上一阶段的思考结果，模型执行一个具体的行动。这可能涉及向环境提问、执行某个命令、或者操作外部系统等，目的是获取新的信息或直接推动任务进展。\n\n3. **Observation（观察）**: 执行行动后，模型会接收并处理来自环境或系统的反馈，这成为新的观察信息。这些观察结果被整合进模型的知识库中，成为下一轮思考的基础。\n\n通过这样的Thought-Action-Observation循环，ReAct框架使AI代理能够逐步逼近任务目标，同时保持决策过程的透明性和可解释性。这种方法不仅提高了模型处理复杂任务的能力，也便于开发者和用户理解和调试模型的行为。然而，它也面临着如输出不稳定、成本高昂及响应时间较长等挑战，这些问题在实际应用中需要通过优化算法或系统设计来缓解。'}]

In [5]:
qwen.get_chat_memory()

[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': 'ReAct是什么意思？'},
 {'role': 'assistant',
  'content': 'ReAct是“Reason and Act”的缩写，这是一种设计智能体（agent）的框架或模式，特别是在基于语言的AI系统中。该框架强调在执行任务时结合推理（Reasoning）与行动（Acting）的能力，旨在让AI系统能够像人类一样，通过思考来指导行动，并根据行动结果进一步调整思考和策略。\n\n具体来说，ReAct框架通常涉及以下几个核心步骤：\n\n1. **观察（Observation）**：智能体接收外界环境或任务的初始信息，作为其行动的起点。\n2. **思考（Thought/Reasoning）**：基于当前的观察和以往的知识，智能体进行推理，决定下一步应该采取什么行动。这个过程中可能包括对目标的分析、计划的制定等。\n3. **行动（Action）**：智能体根据思考的结果执行一个或一系列动作，可能是提问、查询数据库、操作物理世界等。\n4. **反馈（Observation Again）**：行动后，智能体接收新的观察结果，这可以是对之前行动的直接反馈，或是环境状态的更新。\n5. **循环迭代**：智能体基于新的观察继续进行思考和行动，形成一个循环，直至达到解决问题的目标。\n\nReAct框架的优势在于它促进了智能体的透明度和可解释性，因为每一个决策和行动都是基于明确的推理步骤，便于人类理解和调试。同时，这种方法也鼓励了智能体在面对不确定性和复杂性时的适应性和学习能力。然而，它也面临着如成本效率、响应时间和模型稳定性的挑战，正如前面提到的关于LLM（大型语言模型）的一些局限性。'}]

In [6]:
qwen.knowledge

{<illufly.community.faiss.faiss_cpu.FaissDB at 0x1223d4760>,
 Document(text='illufly实现了多智能体结构', metadata='{"source": "unknown"}')}